In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
import math
import random
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.states.simple_rotation_generators import get_binary_x_rotation_provider 
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r, get_g_parameters_provider
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [2]:
generator_layers = 3
discriminator_layers = 5
data_bus_size = 3

In [3]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [4]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [5]:
real, real_symbols = build_x_rotation_state(qubits=data_qubits)

In [6]:
pure_gen = gen.copy()
gen.append([disc])

In [7]:
pure_real = real.copy()
real.append([disc])

In [8]:
print("REAL GROUND STATE")
print(pure_real)

REAL GROUND STATE
(0, 2): ───Rx(r0)───

(0, 3): ───Rx(r1)───

(0, 4): ───Rx(r2)───


In [9]:
print("GENERATOR")
print(pure_gen)

GENERATOR
(0, 2): ───Rx(l)───Rx(g0)───Rz(g4)───ZZ───────────────Rx(g11)───Rz(g15)───ZZ────────────────Rx(g22)───Rz(g26)───ZZ────────────────
                                     │                                    │                                     │
(0, 3): ───Rx(l)───Rx(g1)───Rz(g5)───ZZ^g8───ZZ───────Rx(g12)───Rz(g16)───ZZ^g19───ZZ───────Rx(g23)───Rz(g27)───ZZ^g30───ZZ───────
                                             │                                     │                                     │
(0, 4): ───Rx(l)───Rx(g2)───Rz(g6)───ZZ──────ZZ^g10───Rx(g13)───Rz(g17)───ZZ───────ZZ^g21───Rx(g24)───Rz(g28)───ZZ───────ZZ^g32───
                                     │                                    │                                     │
(0, 5): ───Rx(l)───Rx(g3)───Rz(g7)───ZZ^g9────────────Rx(g14)───Rz(g18)───ZZ^g20────────────Rx(g25)───Rz(g29)───ZZ^g31────────────


In [10]:
print("DISCRIMINATOR")
print(disc)

DISCRIMINATOR
(0, 0): ───Rx(l)───Rx(d0)───Rz(d5)───ZZ────────────────Rx(d14)───Rz(d19)───ZZ────────────────Rx(d28)───Rz(d33)───ZZ────────────────Rx(d42)───Rz(d47)───ZZ────────────────Rx(d56)───Rz(d61)───ZZ────────────────
                                     │                                     │                                     │                                     │                                     │
(0, 1): ───Rx(l)───Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────Rx(d29)───Rz(d34)───ZZ^d38───ZZ───────Rx(d43)───Rz(d48)───ZZ^d52───ZZ───────Rx(d57)───Rz(d62)───ZZ^d66───ZZ───────
                                              │                                     │                                     │                                     │                                     │
(0, 2): ───Rx(l)───Rx(d2)───Rz(d7)───ZZ───────ZZ^d12───Rx(d16)───Rz(d21)───ZZ───────ZZ^d26───Rx(d30)───Rz(d35)───ZZ───────ZZ^d40───Rx(d44)───Rz(d49)───ZZ───────ZZ^d54───Rx(d58)───

In [11]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [12]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [13]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [14]:
g_provider = lambda: random.choice([0, 1, 2])
x_rotations = get_binary_x_rotation_provider({0: '100', 1: '011', 2: '101'})

In [15]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  real_values_provider = x_rotations,
                  use_analytical_expectation=True)

In [16]:
epochs = 101
disc_iteration = 100
gen_iteration = 10
print_interval_epoch = 10
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      print_interval_epoch=print_interval_epoch)

Instructions for updating:
Use fn_output_signature instead
----------------------------------------------------
----------- AFTER DISCRIMINATOR TRAINING -----------
Epoch 0: generator cost = [[-0.349853]]
Epoch 0: discriminator cost = [[0.4387548]]
Prob(fake classified as real):  [[0.349853]]
Prob(real classified as real):  [[0.12418264]]
----------- AFTER GENERATOR TRAINING -----------
Epoch 0: generator cost = [[-0.78478575]]
Epoch 0: discriminator cost = [[0.4413286]]
Prob(fake classified as real):  [[0.34716052]]
Prob(real classified as real):  [[0.7595391]]


KeyboardInterrupt: 

In [59]:
def get_gen_for_g(g, gen_weights):
    rad = map_to_radians(g)
    return np.append(gen_weights, rad)

def get_variables_real(g):
    return np.array([get_theta_v(g), get_theta_w(g), get_theta_r(g)])

def get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, gen_weights, g, size):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights), list(range(size)))
    real = real_evaluator.get_state_from_params(get_variables_real(g))
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, size):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights), list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, ground, cirq.fidelity(generated, ground)

def compare_generated_for_g(gen_evaluator, g1, g2, gen_weights, size):
     generated1 = gen_evaluator.get_state_from_params(get_gen_for_g(g1, gen_weights), list(range(size)))
     generated2 = gen_evaluator.get_state_from_params(get_gen_for_g(g2, gen_weights), list(range(size)))
     return generated1, generated2, cirq.fidelity(generated1, generated2)

In [38]:
trained_disc_weights = tf.Variable(np.array([ 1.25888796e+01,  1.10409822e+01,  1.27487049e+01,  1.32927475e+01,
       -3.20522385e+01,  2.98508596e+00, -7.54223883e-01,  8.97036648e+00,
        8.98472309e+00, -2.77423954e+00,  8.90891266e+00,  5.72837019e+00,
        6.03105211e+00, -4.64482594e+00, -1.10843427e-01,  7.78598115e-02,
        3.00343895e+00,  3.83781940e-01,  6.08641243e+00,  8.64131927e+00,
       -2.17593918e+01,  1.45857897e+01,  2.36893883e+01,  8.75363445e+00,
       -2.42768993e+01,  1.27688437e+01,  2.53628349e+00,  1.39768391e+01,
        1.40961084e+01, -2.04474068e+01,  4.71392822e+00,  1.66926212e+01,
        1.88311214e+01,  9.17525005e+00,  1.16109962e+01,  1.08004580e+01,
        1.10795708e+01,  5.81477690e+00, -5.50215101e+00,  2.22007637e+01,
        2.25015125e+01,  3.21826210e+01, -1.80058708e+01, -7.85126591e+00,
       -7.77073908e+00,  1.40237570e+01,  3.14071465e+01, -1.07477732e+01,
        1.55852342e+00,  2.63786411e+01,  1.31890945e+01,  7.14759350e+00,
        5.46145630e+00, -1.79730053e+01,  7.40563774e+00,  1.06135674e+01,
       -1.55095673e+00, -2.66580944e+01,  6.51995277e+00, -2.37151980e+00,
        3.51896515e+01,  1.50027342e+01,  1.80733763e-02, -1.75052185e+01,
        9.60706902e+00, -6.06413984e+00,  5.35433817e+00, -7.88707399e+00,
        1.27363043e+01,  1.02446747e+01]), dtype=tf.float32)

trained_gen_weights = np.array([  0.83756703,  -8.85081   ,   5.5571547 ,   6.2937136 ,
       -11.173263  ,  -5.2586904 ,  15.195034  ,  -2.752592  ,
        -2.758591  , -11.47981   ,  -5.061309  ,   3.2534666 ,
        -5.6829824 , -22.536098  ,   9.015973  ,   0.5836532 ,
         2.1221178 ,   5.4982877 ,  13.235283  ,   3.4571006 ,
        -0.6003567 ,  -5.9493637 ,  -8.738516  ,   0.4041    ,
         6.9204884 ,  -6.442905  ,  -9.274843  ,  -3.695387  ,
        10.760881  ,   2.24611   ,   0.5289249 ,  14.787433  ,
       -11.227607])

gen_symbols = gs + (ls,)
real_symbols = ('r0', 'r1', 'r2')

In [39]:
gen_evaluator = CircuitEvaluator(pure_gen, gen_symbols)
real_evaluator = CircuitEvaluator(pure_real, real_symbols)

In [66]:
g = -10
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights, g, data_bus_size)

Max probability state after tracing has probability: 0.974941611289978


(array([-0.08993246+0.j        ,  0.38960975-0.05731301j,
         0.08879194+0.24945204j,  0.17451479-0.65830445j,
         0.05236622-0.05087227j, -0.14689884+0.10184895j,
        -0.07625695+0.15308213j,  0.43438524-0.21835442j], dtype=complex64),
 array([ 0.11732721+0.j        ,  0.        -0.03710212j,
         0.        -0.76015157j, -0.24038103+0.j        ,
         0.        -0.08595169j, -0.02718031+0.j        ,
        -0.55687267+0.j        ,  0.        +0.1760986j ], dtype=complex64),
 0.05823949895534164)

In [67]:
g = 10
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights, g, data_bus_size)

Max probability state after tracing has probability: 0.974941611289978


(array([-0.08993246+0.j        ,  0.38960975-0.05731301j,
         0.08879194+0.24945204j,  0.17451479-0.65830445j,
         0.05236622-0.05087227j, -0.14689884+0.10184895j,
        -0.07625695+0.15308213j,  0.43438524-0.21835442j], dtype=complex64),
 array([ 0.76015157+0.j        ,  0.        -0.24038103j,
         0.        -0.11732721j, -0.03710212+0.j        ,
         0.        -0.55687267j, -0.1760986 +0.j        ,
        -0.08595169+0.j        ,  0.        +0.02718031j], dtype=complex64),
 0.014430476418150429)

In [68]:
compare_generated_for_g(gen_evaluator, -9, 9, trained_gen_weights, data_bus_size)

Max probability state after tracing has probability: 0.9781550168991089
Max probability state after tracing has probability: 0.981857419013977


(array([-0.0841965 +0.j        ,  0.18691188+0.1448713j ,
        -0.04824434-0.12588938j, -0.69796413+0.39543998j,
         0.09763314+0.01778871j, -0.23618338-0.27832115j,
         0.03212571-0.0409976j , -0.33659515-0.12720747j], dtype=complex64),
 array([-0.0923465 +0.0000000e+00j, -0.07581477-4.6684846e-01j,
         0.08082028-1.9660823e-01j,  0.20732668+6.3416529e-01j,
         0.01657245+1.6675850e-02j, -0.07306691+1.8806756e-01j,
         0.14650305-2.5829076e-04j, -0.3853701 +2.5730491e-01j],
       dtype=complex64),
 0.6142508851638837)

In [35]:
g = 1
get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, data_bus_size)

Max probability state after tracing has probability: 0.981857419013977


(array([-0.0923465 +0.0000000e+00j, -0.07581477-4.6684846e-01j,
         0.08082028-1.9660823e-01j,  0.20732668+6.3416529e-01j,
         0.01657245+1.6675850e-02j, -0.07306691+1.8806756e-01j,
         0.14650305-2.5829076e-04j, -0.3853701 +2.5730491e-01j],
       dtype=complex64),
 array([0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
        0.35355339, 0.35355339, 0.35355339]),
 0.027226466798719495)